In [11]:
import argparse

from point_e.evals.feature_extractor import PointNetClassifier, get_torch_devices
from point_e.evals.fid_is import compute_statistics, compute_inception_score
from point_e.evals.npz_stream import NpzStreamer

import torch
import numpy as np
import os

In [12]:
def npy2npz(data_dir, save_path):
    file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.npy')]
    point_clouds = np.stack([np.load(file) for file in file_list])
    np.savez(save_path, point_clouds)

In [13]:
clf = PointNetClassifier(devices=get_torch_devices())

/home/anguyenk/Documents/point-e/point_e/models/download.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)


In [6]:
ptv3_pcs = torch.load('/home/anguyenk/Documents/adaLN_epoch1599_grad_clip1_no_flash_attn_drop_010_grid_size_010_patch_size_64/samples.pth')
np.savez('/home/anguyenk/Documents/adaLN_epoch1599_grad_clip1_no_flash_attn_drop_010_grid_size_010_patch_size_64/samples.npz', ptv3_pcs.numpy())

/tmp/ipykernel_6560/1749426100.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ptv3_pcs = torch.load('/home/anguyenk/Documents/adaLN_epoch1599_grad_clip1_no_flash_attn_d

In [14]:
features_ptv3, preds_ptv3 = clf.features_and_preds(NpzStreamer('/home/anguyenk/Documents/adaLN_epoch1599_grad_clip1_no_flash_attn_drop_010_grid_size_010_patch_size_64/samples.npz'))
features_pvd, preds_pvd = clf.features_and_preds(NpzStreamer('/home/anguyenk/Documents/point-e/point_e/examples/PVD_checkpoint_chairs.npz'))
features_val, preds_val = clf.features_and_preds(NpzStreamer('/home/anguyenk/Documents/point-e/point_e/examples/val_chairs.npz'))

In [15]:
stats_ptv3 = compute_statistics(features_ptv3)
stats_pvd = compute_statistics(features_pvd)
stats_val = compute_statistics(features_val)

In [24]:
print(f"P-FID: {stats_pvd.frechet_distance(stats_val)}")

P-FID: 13.822413519173551


In [25]:
print(f"P-IS: {compute_inception_score(preds_pvd)}")

P-IS: 2.6209261417388916


In [26]:
print(f"P-FID: {stats_ptv3.frechet_distance(stats_val)}")

P-FID: 17.287379322255845


In [27]:
print(f"P-IS: {compute_inception_score(preds_ptv3)}")

P-IS: 2.6711742877960205


In [22]:
print(f"P-FID: {stats_ptv3.frechet_distance(stats_pvd)}")

P-FID: 5.248544493347737


In [23]:
print(f"P-FID: {stats_pvd.frechet_distance(stats_ptv3)}")

P-FID: 5.248544493584291
